In [8]:
import altair as alt
import pandas as pd
import numpy as np
from covid import dataimport
from covid import chart

In [2]:
fhm_data, labels = dataimport.get_fhm_data()

In [3]:
df = fhm_data[fhm_data['date'] >= '2020-03-13']
chart.lag_chart_with_selection(df, labels)

alt.HConcatChart(...)

In [4]:
df = fhm_data
lag_chart = chart.average_lag(df, '2020-04-03')
lag_chart

alt.Chart(...)

In [5]:
df = fhm_data.dropna(subset=['days_since_publication'])
df = df[df.date >= '2020-04-06'] # Start on first relevant Monday

chart.daily_reported_deaths(df, labels)

alt.FacetChart(...)

# Work in progress
Experimental stuff

In [6]:
df = fhm_data[fhm_data['date'] > '2020-03-10']
# Only show weekly publication dates based on latest publication date
df = df[df.publication_date.dt.dayofweek == fhm_data['publication_date'].max().weekday()]

alt.Chart(df, width=600).mark_line(interpolate='basis').encode(
    x=alt.X('date', title="Date"),
    y=alt.Y('N', title="Deceased"),
    color=alt.Color('monthdate(publication_date):N', title="Publication Date")
)

alt.Chart(...)

In [9]:
df = fhm_data[fhm_data['publication_date'] >= '2020-04-03']
df = df.replace(0, np.nan)

alt.Chart(df,height=200, width=600).mark_rect().encode(
    x='yearmonthdate(publication_date)',
#    y='yearmonthdate(date)',
    y=alt.Y('lag', sort=labels),
    size='n_diff',
    color=alt.Color('n_diff', title='Deaths', scale=alt.Scale(scheme='goldgreen')),
    tooltip=[alt.Tooltip('n_diff', title='Deaths')]
)

alt.Chart(...)